In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

import matplotlib.pyplot as plt

In [72]:
FOLDS = 3

def calculateF1Scores(df, label):
    f1score_only_above05 = []
    prec = []
    rec = []
    seed_list = []

    for seed in range(FOLDS):
        df_part = df[df["seed"] == seed].copy()
        f1score_1 = f1_score(df_part[label],
                                   np.where(df_part["pred"].values >= 0.5, 1, 0), zero_division=0)
        recall = recall_score(df_part[label],
                                   np.where(df_part["pred"].values >= 0.5, 1, 0))
        precision = precision_score(df_part[label],
                                   np.where(df_part["pred"].values >= 0.5, 1, 0))
        
        f1score_only_above05.append(f1score_1)
        rec.append(recall)
        prec.append(precision)

        seed_list.append(seed)

    f1_scores = pd.DataFrame({
    'seed': seed_list,
    'f1score': f1score_only_above05,
    'recall': rec,
    'precision': prec        
        
    })

    return f1_scores

def calcF1_multiClass(df, allClasses, prediction):
    f1 = []
    for seed in range(FOLDS):
        df_part = df[df["seed"] == seed].copy()
        f1.append(f1_score(df_part[allClasses], df_part[[prediction + c for c in allClasses]], zero_division=0, average='macro'))
    
    return f1


### Relevance

In [74]:
bert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-04-06_test_predicted_relevant_Hyperparam_3fold.csv")
huberSDG = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-05_test_relevant_predicted_sklearn.csv") 
distilbert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-04_test_predicted_relevant_distilbert.csv") 

bert_f1 = calculateF1Scores(bert, 'label_3')[:3]
distil_f1 = calculateF1Scores(distilbert, 'label_3')[:3]
sdg_f1 = calculateF1Scores(huberSDG, 'label_3')[:3]

In [77]:
print('F1 mean')
print(bert_f1.f1score.mean())
print(distil_f1.f1score.mean())
print(sdg_f1.f1score.mean())
print('F1 stdv')
print(bert_f1.f1score.std())
print(distil_f1.f1score.std())
print(sdg_f1.f1score.std())
print('recall')
print(bert_f1.recall.mean())
print(bert_f1.recall.std())
print('precision')
print(bert_f1.precision.mean())
print(bert_f1.precision.std())


F1 mean
0.9068482684526505
0.8974247544038305
0.8767964917647023
F1 stdv
0.0075669546807272045
0.007413216446246565
0.005986364125556633
recall
0.9340233085184364
0.00482928192947384
precision
0.8814055699103509
0.018515303998170876


### Technology

In [68]:
bert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-03-20_test_predicted_tech_unbalanced_hyperparamTuning.csv")
distilbert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-05_test_predicted_tech_distilbert.csv") 
huberSDG = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-05_test_tech_predicted_sklearn.csv") 

allClasses = [i.split("predOptimal_")[1] for i in bert.columns if "predOptimal" in i]

bert_f1 = calcF1_multiClass(bert, allClasses,"pred_")
bert_opt_f1 = calcF1_multiClass(bert, allClasses,"predOptimal_")
distil_f1 = calcF1_multiClass(distilbert, allClasses,"pred_")
sdg_f1 = calcF1_multiClass(huberSDG, allClasses,"pred_")

In [69]:
print('F1 mean')
print(np.mean(bert_f1))
print(np.mean(bert_opt_f1))
print(np.mean(distil_f1))
print(np.mean(sdg_f1))
print('F1 stdv')
print(np.std(bert_f1))
print(np.std(bert_opt_f1))
print(np.std(distil_f1))
print(np.std(sdg_f1))

F1 mean
0.7110780534534507
0.7663630625992921
0.5296153066278994
0.689104781223617
F1 stdv
0.08569219004098673
0.07403815973533164
0.00980357681337249
0.006215094002978524


### Methods

In [62]:
bert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-04-11_test_meth_predicted_unbalanced_hyperparamTuning_3fold.csv")
distilbert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-05_test_predicted_methods_distilbert.csv") 
huberSDG = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-05_test_meth_predicted_sklearn.csv") 

allClasses_old = [i.split("predOptimal_")[1] for i in bert.columns if "predOptimal" in i]
allClasses_new = [i.split("predProba_")[1] for i in distilbert.columns if "predProba" in i]


bert_f1 = calcF1_multiClass(bert, allClasses_old,"pred_")
bert_opt_f1 = calcF1_multiClass(bert, allClasses_old,"predOptimal_")
distil_f1 = calcF1_multiClass(distilbert, allClasses_new,"pred_")
sdg_f1 = calcF1_multiClass(huberSDG, allClasses_new,"pred_")

In [63]:
print('F1 mean')
print(np.mean(bert_f1))
print(np.mean(bert_opt_f1))
print(np.mean(distil_f1))
print(np.mean(sdg_f1))
print('F1 stdv')
print(np.std(bert_f1))
print(np.std(bert_opt_f1))
print(np.std(distil_f1))
print(np.std(sdg_f1))

F1 mean
0.6433859082107366
0.6741652830385659
0.389035833784577
0.4911498207572374
F1 stdv
0.11562016286453528
0.1282143828077215
0.01593315027787941
0.04283076244847008


### Context

In [70]:
bert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-03-20_test_predicted_tech_unbalanced_hyperparamTuning.csv")
distilbert = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-05_test_predicted_tech_distilbert.csv") 
huberSDG = pd.read_csv("/home/sarah/projects/nets_climatebert/data/2023-07-05_test_tech_predicted_sklearn.csv") 

allClasses = [i.split("predOptimal_")[1] for i in bert.columns if "predOptimal" in i]

bert_f1 = calcF1_multiClass(bert, allClasses,"pred_")
bert_opt_f1 = calcF1_multiClass(bert, allClasses,"predOptimal_")
distil_f1 = calcF1_multiClass(distilbert, allClasses,"pred_")
sdg_f1 = calcF1_multiClass(huberSDG, allClasses,"pred_")

In [71]:
print('F1 mean')
print(np.mean(bert_f1))
print(np.mean(bert_opt_f1))
print(np.mean(distil_f1))
print(np.mean(sdg_f1))
print('F1 stdv')
print(np.std(bert_f1))
print(np.std(bert_opt_f1))
print(np.std(distil_f1))
print(np.std(sdg_f1))

F1 mean
0.7110780534534507
0.7663630625992921
0.5296153066278994
0.689104781223617
F1 stdv
0.08569219004098673
0.07403815973533164
0.00980357681337249
0.006215094002978524
